<a href="https://www.kaggle.com/code/akshaypatil7015/used-car-price-prediction-main?scriptVersionId=173199114" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-cars-price-prediction/test-data.csv
/kaggle/input/used-cars-price-prediction/train-data.csv


In [2]:
df=pd.read_csv('/kaggle/input/used-cars-price-prediction/train-data.csv')
df_test=pd.read_csv('/kaggle/input/used-cars-price-prediction/test-data.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [3]:
df_test.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,NaN
1,1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,NaN
2,2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,25.27 Lakh
3,3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,NaN
4,4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6019 non-null   int64  
 1   Name               6019 non-null   object 
 2   Location           6019 non-null   object 
 3   Year               6019 non-null   int64  
 4   Kilometers_Driven  6019 non-null   int64  
 5   Fuel_Type          6019 non-null   object 
 6   Transmission       6019 non-null   object 
 7   Owner_Type         6019 non-null   object 
 8   Mileage            6017 non-null   object 
 9   Engine             5983 non-null   object 
 10  Power              5983 non-null   object 
 11  Seats              5977 non-null   float64
 12  New_Price          824 non-null    object 
 13  Price              6019 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 658.5+ KB


In [5]:
df.describe()

,Unnamed: 0,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,3009.000000,2013.358199,5.873838e+04,5.278735,9.479468
std,1737.679967,3.269742,9.126884e+04,0.808840,11.187917
min,0.000000,1998.000000,1.710000e+02,0.000000,0.440000
25%,1504.500000,2011.000000,3.400000e+04,5.000000,3.500000
50%,3009.000000,2014.000000,5.300000e+04,5.000000,5.640000
75%,4513.500000,2016.000000,7.300000e+04,5.000000,9.950000
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000


In [6]:
name=df['Name'].str.split(n=3, expand=True)
df['Company']=name[0]
df['model']=name[1]
df.drop(['Unnamed: 0','New_Price','Location','Name'], axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,model
0,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75,Maruti,Wagon
1,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50,Hyundai,Creta
2,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Honda,Jazz
3,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00,Maruti,Ertiga
4,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74,Audi,A4


In [7]:
df.isna().sum()

Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
Company              0
model                0
dtype: int64

In [8]:
df['Mileage'] = df['Mileage'].str.replace('km/kg', '')
df['Mileage'] = df['Mileage'].str.replace('kmpl', '')
df['Engine'] = df['Engine'].str.replace('CC', '')
df['Power'] = df['Power'].str.replace('bhp', '')

In [9]:
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')

In [10]:
df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,model
0,2010,72000,CNG,Manual,First,26.60,998,58.16,5.0,1.75,Maruti,Wagon
1,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50,Hyundai,Creta
2,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50,Honda,Jazz
3,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00,Maruti,Ertiga
4,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74,Audi,A4


In [11]:
df['Power'].fillna(df['Power'].mean(), inplace=True)

df.isnull().sum()

/tmp/ipykernel_18/4263623107.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Power'].fillna(df['Power'].mean(), inplace=True)


Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
Company              0
model                0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
le = LabelEncoder()

df['Fuel_Type']=le.fit_transform(df['Fuel_Type'])
df['Transmission']=le.fit_transform(df['Transmission'])

ordinal_map = {'First':1, 'Second':2, 'Third':3, 'Fourth & Avove':4}
df['Owner_Type']=df['Owner_Type'].map(ordinal_map)
df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,model
0,2010,72000,0,1,1.0,26.60,998,58.16,5.0,1.75,Maruti,Wagon
1,2015,41000,1,1,1.0,19.67,1582,126.20,5.0,12.50,Hyundai,Creta
2,2011,46000,3,1,1.0,18.20,1199,88.70,5.0,4.50,Honda,Jazz
3,2012,87000,1,1,1.0,20.77,1248,88.76,7.0,6.00,Maruti,Ertiga
4,2013,40670,1,0,2.0,15.20,1968,140.80,5.0,17.74,Audi,A4


In [13]:
df=pd.get_dummies(df, drop_first=True, dtype='int', columns=['Company', 'model'])

In [14]:
df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,...,model_Xcent,model_Xenon,model_Xylo,model_Yeti,model_Z4,model_Zen,model_Zest,model_i10,model_i20,model_redi-GO
0,2010,72000,0,1,1.0,26.60,998,58.16,5.0,1.75,...,0,0,0,0,0,0,0,0,0,0
1,2015,41000,1,1,1.0,19.67,1582,126.20,5.0,12.50,...,0,0,0,0,0,0,0,0,0,0
2,2011,46000,3,1,1.0,18.20,1199,88.70,5.0,4.50,...,0,0,0,0,0,0,0,0,0,0
3,2012,87000,1,1,1.0,20.77,1248,88.76,7.0,6.00,...,0,0,0,0,0,0,0,0,0,0
4,2013,40670,1,0,2.0,15.20,1968,140.80,5.0,17.74,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.dropna(inplace=True)
df.isna().sum()

Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
                    ..
model_Zen            0
model_Zest           0
model_i10            0
model_i20            0
model_redi-GO        0
Length: 248, dtype: int64

## PCA

In [16]:
X=df.drop('Price', axis=1)
y=df[['Price']]
print(X.shape, y.shape)

(5967, 247) (5967, 1)


In [17]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_standardized = std_scaler.fit_transform(X)

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5, random_state=21)
X_pca = pca.fit_transform(X_standardized)
X_pca_df=pd.DataFrame(X_pca)
X_pca_df.rename(columns={0:'PCA_1', 1: 'PCA_2', 2: 'PCA_3', 3: 'PCA_4', 4: 'PCA_5'}, inplace=True)
X_pca_df

,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5
0,-2.333415,-0.308257,-1.834022,1.531023,0.216916
1,-0.527001,0.356259,0.003700,-2.666672,0.451113
2,-1.301116,0.674786,1.898228,0.696959,-2.333514
3,-1.051705,-2.051664,-1.994144,1.032258,-0.164594
4,2.647650,1.665469,-0.476300,-0.135030,1.048755
...,...,...,...,...,...
5962,-2.317224,-0.086621,-2.461261,1.074177,-0.027774
5963,-1.858084,0.198363,-0.242877,-2.568480,0.598148
5964,2.393022,-5.516046,-0.192568,0.238442,-0.226936
5965,-2.308157,0.056193,-0.828185,1.388683,-0.193892


# Model Building

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 31)

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4773, 247) (1194, 247) (4773, 1) (1194, 1)


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
lin_mod = LinearRegression()
ran_mod = RandomForestRegressor()

In [22]:
lin_mod.fit(X_train, y_train)
ran_mod.fit(X_train, y_train)

lin_pred = lin_mod.predict(X_test)
ran_pred = ran_mod.predict(X_test)

/tmp/ipykernel_18/794179974.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ran_mod.fit(X_train, y_train)


In [23]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

lin_rmse = sqrt(mean_squared_error(lin_pred, y_test))
lin_r2 = r2_score(lin_pred, y_test)

ran_rmse = sqrt(mean_squared_error(ran_pred, y_test))
ran_r2 = r2_score(ran_pred, y_test)

## After PCA

In [24]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca_df, y, test_size = 0.2, random_state = 41)

lin_mod_pca = LinearRegression()
ran_mod_pca = RandomForestRegressor()

In [25]:
lin_mod_pca.fit(X_train_pca, y_train_pca)
ran_mod_pca.fit(X_train_pca, y_train_pca)

lin_pred_pca = lin_mod_pca.predict(X_test_pca)
ran_pred_pca = ran_mod_pca.predict(X_test_pca)


lin_rmse_pca = sqrt(mean_squared_error(lin_pred_pca, y_test_pca))
lin_r2_pca = r2_score(lin_pred_pca, y_test_pca)

ran_rmse_pca = sqrt(mean_squared_error(ran_pred_pca, y_test_pca))
ran_r2_pca = r2_score(ran_pred_pca, y_test_pca)

/tmp/ipykernel_18/960291428.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ran_mod_pca.fit(X_train_pca, y_train_pca)


In [26]:
print(f'Before PCA linear model r2 = {lin_r2} and RMSE = {lin_rmse}')
print(f'After PCA linear model r2 = {lin_r2_pca} and RMSE = {lin_rmse_pca}')
print()
print(f'Before PCA random forest model r2 = {ran_r2} and RMSE = {ran_rmse}')
print(f'After PCA random forest model r2 = {ran_r2_pca} and RMSE = {ran_rmse_pca}')

Before PCA linear model r2 = 0.685929690382252 and RMSE = 6.089875200874569
After PCA linear model r2 = 0.5239024897676685 and RMSE = 6.421089179833452

Before PCA random forest model r2 = 0.8882296167206448 and RMSE = 3.595374591737088
After PCA random forest model r2 = 0.8468824583699842 and RMSE = 4.174826278589234


In [27]:
from sklearn.metrics import get_scorer_names
get_scorer=get_scorer_names()
get_scorer

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [28]:
from sklearn.model_selection import cross_val_score

lin_cv_score = cross_val_score(lin_mod, X_train, y_train, cv=7, scoring='r2')
ran_cv_score = cross_val_score(ran_mod, X_train, y_train, cv=7, scoring='r2')

lin_cv_score_pca = cross_val_score(lin_mod_pca, X_train_pca, y_train_pca, cv=7, scoring='r2')
ran_cv_score_pca = cross_val_score(ran_mod_pca, X_train_pca, y_train_pca, cv=7, scoring='r2')

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

In [29]:
data={'model':['lin_cv_score', 'ran_cv_score', 'lin_cv_score_pca', 'ran_cv_score_pca'], 
      'value':[lin_cv_score.mean(), ran_cv_score.mean(), lin_cv_score_pca.mean(),ran_cv_score_pca.mean()]}

In [30]:
df_cv=pd.DataFrame(data)
df_cv

,model,value
0,lin_cv_score,0.812659
1,ran_cv_score,0.887987
2,lin_cv_score_pca,0.664366
3,ran_cv_score_pca,0.869046
